## Part 1: Criando e Manipulando RDDs

### (1a) Criando uma base RDD 

In [2]:
ListaPalavras = ['gato', 'elefante', 'rato', 'rato', 'gato']
palavrasRDD = sc.parallelize(ListaPalavras, 4)
print(type(palavrasRDD))

<class 'pyspark.rdd.RDD'>


### (1b) Plural

In [14]:
# EXERCICIO
def Plural(palavra):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    P="s"
    return palavra+"{}".format(P)

print(Plural('gato'))

gatos


In [6]:
help(Plural)

Help on function Plural in module __main__:

Plural(palavra)
    Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
    
    Returns:
        str: A string with 's' added to it.



In [22]:
assert Plural('rato')=='ratos', 'resultado incorreto!'
print ('OK')

OK


### (1c) Aplicando a função ao RDD

In [31]:
# EXERCICIO
pluralRDD = palavrasRDD.map(lambda x:(Plural(x)))
print (pluralRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [25]:
assert pluralRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print ('OK')

OK


### (1d) Utilizando uma função lambda

In [86]:
# EXERCICIO
pluralLambdaRDD = palavrasRDD.map(lambda x:(Plural(x)))
print (pluralLambdaRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [28]:
assert pluralLambdaRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print ('OK')

OK


### (1e) Tamanho de cada palavra

In [93]:
# EXERCICIO
pluralTamanho = (pluralRDD
                 .map(lambda x:(len(x))) 
                 .collect()
                 )
print (pluralTamanho)

[5, 9, 5, 5, 5]


In [94]:
assert pluralTamanho==[5,9,5,5,5], 'valores incorretos'
print ("OK")

OK


### (1f) RDDs de pares e tuplas

In [131]:
# EXERCICIO
palavraPar = palavrasRDD.map(lambda x:(x,1))
print (palavraPar.collect())

[('gato', 1), ('elefante', 1), ('rato', 1), ('rato', 1), ('gato', 1)]


In [132]:
assert palavraPar.collect() == [('gato',1),('elefante',1),('rato',1),('rato',1),('gato',1)], 'valores incorretos!'
print ("OK")

OK


## Parte 2

### (2a) Função groupByKey()

In [148]:
def f(chave,valores):
    palavrasGrupo.mapValues(lambda x: list(x)).collect()
    return (chave,valores)

In [149]:
# EXERCICIO
palavrasGrupo = palavraPar.groupByKey()
for chave, valor in palavrasGrupo.collect():
    valores = list(valor)
    print(f(chave,valores))

('rato', [1, 1])
('elefante', [1])
('gato', [1, 1])


In [150]:
assert sorted(palavrasGrupo.mapValues(lambda x: list(x)).collect()) == [('elefante', [1]), ('gato',[1, 1]), ('rato',[1, 1])], 'Valores incorretos!'
print ("OK")

OK


### 2b) Calculando as contagems

In [172]:
# EXERCICIO
contagemGroup = palavrasGrupo.mapValues(lambda x:sum(x))
print (contagemGroup.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [173]:
assert sorted(contagemGroup.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


### 2c) reduceByKey

In [176]:
# EXERCICIO
contagem = palavraPar.reduceByKey(add)
print( contagem.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [177]:
assert sorted(contagem.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


### (2d) Agrupando os comandos

In [197]:
# EXERCICIO
contagemFinal = (palavrasRDD
                 .map(lambda x:(x,1))
                 .reduceByKey(add)                
                 )
print (contagemFinal.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [198]:
assert sorted(contagemFinal.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


## Parte 3: Encontrando as palavras únicas e calculando a média de contagem 

### (3a) Palavras Únicas

In [200]:
# EXERCICIO
palavrasUnicas = (contagemFinal.count()
                 )
print (palavrasUnicas)

3


In [201]:
assert palavrasUnicas==3, 'valor incorreto!'
print ("OK")

OK


### (3b) Calculando a Média de contagem de palavras

In [221]:
# EXERCICIO
from operator import add
total = (contagemFinal
         .map(lambda x:x[1])
         .reduce(add) 
         )
media = total / float(palavrasUnicas)
print (total)
print (round(media, 2))

5
1.67


In [220]:
assert round(media, 2)==1.67, 'valores incorretos!'
print ("OK")

OK


## Parte 4: Aplicar nosso algoritmo em um arquivo 

### (4a) Função contaPalavras


In [226]:
# EXERCICIO
def contaPalavras(chavesRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        chavesRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return (chavesRDD
            .map(lambda x:(x,1))
            .reduceByKey(add)
           )

print (contaPalavras(palavrasRDD).collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [227]:
assert sorted(contaPalavras(palavrasRDD).collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


### (4b) Normalizando o texto

In [228]:
# EXERCICIO
import re
def removerPontuacao(texto):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        texto (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()
print (removerPontuacao('Ola, quem esta ai??!'))
print (removerPontuacao(' Sem espaco e_sublinhado!'))

ola quem esta ai
sem espaco esublinhado


In [229]:
assert removerPontuacao(' O uso de virgulas, embora permitido, nao deve contar. ')=='o uso de virgulas embora permitido nao deve contar', 'string incorreta!'
print ("OK")

OK


### (4c) Carregando arquivo texto 

In [232]:
# Apenas execute a célula
import os.path

arquivo = os.path.join('Data', 'pg100.txt') 

# lê o arquivo com textFile e aplica a função removerPontuacao        
shakesRDD = (sc
             .textFile(arquivo, 8)
             .map(removerPontuacao)
             )

# zipWithIndex gera tuplas (conteudo, indice) onde indice é a posição do conteudo na lista sequencial
# Ex.: sc.parallelize(['gato','cachorro','boi']).zipWithIndex() ==> [('gato',0), ('cachorro',1), ('boi',2)]
# sep.join() junta as strings de uma lista através do separador sep. Ex.: ','.join(['a','b','c']) ==> 'a,b,c'
print ('\n'.join(shakesRDD
                .zipWithIndex()
                .map(lambda linha: '{0}: {1}'.format(linha[0],linha[1]))
                .take(15)
               ))

: 0
project gutenbergs the complete works of william shakespeare by william: 1
shakespeare: 2
: 3
this ebook is for the use of anyone anywhere in the united states and: 4
most other parts of the world at no cost and with almost no restrictions: 5
whatsoever  you may copy it give it away or reuse it under the terms: 6
of the project gutenberg license included with this ebook or online at: 7
wwwgutenbergorg  if you are not located in the united states youll: 8
have to check the laws of the country where you are located before using: 9
this ebook: 10
: 11
see at the end of this file  content note added in 2017: 12
: 13
: 14


### (4d) Extraindo as palavras 

In [246]:
# EXERCICIO
shakesPalavrasRDD = shakesRDD.map(lambda x:x.split(" "))
total = shakesPalavrasRDD.count()
print (shakesPalavrasRDD.take(5))
print (total)

[[u''], [u'project', u'gutenbergs', u'the', u'complete', u'works', u'of', u'william', u'shakespeare', u'by', u'william'], [u'shakespeare'], [u''], [u'this', u'ebook', u'is', u'for', u'the', u'use', u'of', u'anyone', u'anywhere', u'in', u'the', u'united', u'states', u'and']]
147929


### (4e) Total sem espacos

In [249]:
# EXERCICIO
shakesPalavrasRDD = shakesRDD.flatMap(lambda x: x.split())
total = shakesPalavrasRDD.count()
print (shakesPalavrasRDD.take(5))
print (total)

[u'project', u'gutenbergs', u'the', u'complete', u'works']
959359


In [250]:
assert total==959359, "valor incorreto de palavras!"
print ("OK")
assert shakesPalavrasRDD.take(5)==['project', 'gutenbergs', 'the', 'complete', 'works'],'lista incorreta de palavras'
print ("OK")


OK
OK


### (4f) Contagem de palavras

In [310]:
# EXERCICIO
Palavras_con_Cantidad=contaPalavras(shakesPalavrasRDD)

#.takeOrdered(15,key=lambda x:-x)
top15 =  shakesPalavrasRDD.takeOrdered(15,key=lambda x: x)
#print Palavras_con_Cantidad.values().collect()
print ('\n'.join(map(lambda x : x,Palavras_con_Cantidad.keys().collect())))

fawn
voluble
divinely
signories
mustachio
resist
shardborne
joshua
four
seeabout
bere
mansion
deceives
consumptions
friendsa
adown
looking
unkindest
banishd
gelidus
propounded
tyde
presents
nunnery
buyer
buttonhole
somever
sinking
entertaind
deluded
seatost
tinctures
impertinent
monachum
compassion
treacherie
twentysix
morrisse
foreward
jests
shoot
unloving
bearward
himdo
breakpromise
swineherds
omitst
boarshead
suis
knowest
bravest
latchd
mastercord
wilfull
wooden
deficient
regenerate
sulphrous
abatements
montague
sacks
216
particularities
wanderers
slipper
enjoy
achievements
tinvite
oertrip
oerwhelming
blasphemy
smeared
embracement
honorificabilitudinitatibus
chaplains
tires
seald
beech
consenting
flybitten
billets
impugns
wayes
even
lesser
untoward
sevnight
solemn
pyniond
windowes
heire
prisond
pours
devotions
symbols
ebbs
cannibals
waies
1e1
hinge
overripend
nors
warre
impartial
pancakes
interrupt
fatter
chimneys
chine
weepst
coold
ment
sorceress
unattempted
truththe
fourteen
maim


In [311]:
assert top15 == [('the', 29996), ('and', 28353), ('i', 21860), ('to', 20885), ('of', 18811), ('a', 15992), ('you', 14439), ('my', 13191), ('in', 12027), ('that', 11782), ('is', 9711), ('not', 9068), ('with', 8521), ('me', 8271), ('for', 8184)],'valores incorretos!'
print ("OK")

AssertionError: valores incorretos!

## Parte 5: Similaridade entre Objetos

### (5a) Vetores no espaço Euclidiano

In [11]:
import numpy as np

# Vamos criar uma função pNorm que recebe como parâmetro p e retorna uma função que calcula a pNorma
def pNorm(p):
    """Generates a function to calculate the p-Norm between two points.

    Args:
        p (int): The integer p.

    Returns:
        Dist: A function that calculates the p-Norm.
    """

    def Dist(x,y):
        return np.power(np.power(np.abs(x-y),p).sum(),1/float(p))
    return Dist

In [12]:
# Vamos criar uma RDD com valores numéricos
np.random.seed(42)
numPointsRDD = sc.parallelize(enumerate(np.random.random(size=(10,100))))


In [18]:
print numPointsRDD.collect()

[(0, array([ 0.37454012,  0.95071431,  0.73199394,  0.59865848,  0.15601864,
        0.15599452,  0.05808361,  0.86617615,  0.60111501,  0.70807258,
        0.02058449,  0.96990985,  0.83244264,  0.21233911,  0.18182497,
        0.18340451,  0.30424224,  0.52475643,  0.43194502,  0.29122914,
        0.61185289,  0.13949386,  0.29214465,  0.36636184,  0.45606998,
        0.78517596,  0.19967378,  0.51423444,  0.59241457,  0.04645041,
        0.60754485,  0.17052412,  0.06505159,  0.94888554,  0.96563203,
        0.80839735,  0.30461377,  0.09767211,  0.68423303,  0.44015249,
        0.12203823,  0.49517691,  0.03438852,  0.9093204 ,  0.25877998,
        0.66252228,  0.31171108,  0.52006802,  0.54671028,  0.18485446,
        0.96958463,  0.77513282,  0.93949894,  0.89482735,  0.59789998,
        0.92187424,  0.0884925 ,  0.19598286,  0.04522729,  0.32533033,
        0.38867729,  0.27134903,  0.82873751,  0.35675333,  0.28093451,
        0.54269608,  0.14092422,  0.80219698,  0.07455064, 

In [21]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartPointsRDD = numPointsRDD.cartesian(numPointsRDD)

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartPointsParesRDD = cartPointsRDD.map(lambda x,y:x,y=len(x)).take(1)

print cartPointsParesRDD


NameError: name 'x' is not defined

In [315]:
assert (minv.round(2), maxv.round(2), meanv.round(2))==(0.0, 4.71, 3.75), 'Valores incorretos'
print ("OK")

NameError: name 'minv' is not defined

### (5b) Valores Categóricos 

In [ ]:
# Vamos criar uma função para calcular a distância de Hamming
def Hamming(x,y):
    """Calculates the Hamming distance between two binary vectors.

    Args:
        x, y (np.array): Array of binary integers x and y.

    Returns:
        H (int): The Hamming distance between x and y.
    """
    return (x!=y).sum()

# Vamos criar uma função para calcular a distância de Jaccard
def Jaccard(x,y):
    """Calculates the Jaccard distance between two binary vectors.

    Args:
        x, y (np.array): Array of binary integers x and y.

    Returns:
        J (int): The Jaccard distance between x and y.
    """
    return (x==y).sum()/float( np.maximum(x,y).sum() )


In [316]:
# Vamos criar uma RDD com valores categóricos
catPointsRDD = sc.parallelize(enumerate([['alto', 'caro', 'azul'],
                             ['medio', 'caro', 'verde'],
                             ['alto', 'barato', 'azul'],
                             ['medio', 'caro', 'vermelho'],
                             ['baixo', 'barato', 'verde'],
                            ]))

In [ ]:
# EXERCICIO
# Crie um RDD de chaves únicas utilizando flatMap
chavesRDD = (catPointsRDD
             .<COMPLETAR>
             .<COMPLETAR>
             .<COMPLETAR>
             )

chaves = dict((v,k) for k,v in chavesRDD.collect())
nchaves = len(chaves)
print (chaves, nchaves)

In [ ]:
assert chaves=={'alto': 2, 'caro': 0, 'baixo': 4, 'verde': 1, 'azul': 2, 'medio': 3, 'barato': 4, 'vermelho': 3}, 'valores incorretos!'
print ("OK")

assert nchaves==8, 'número de chaves incorreta'
print ("OK")

In [ ]:
def CreateNP(atributos,chaves):  
    """Binarize the categorical vector using a dictionary of keys.

    Args:
        atributos (list): List of attributes of a given object.
        chaves (dict): dictionary with the relation attribute -> index

    Returns:
        array (np.array): Binary array of attributes.
    """
    
    array = np.zeros(len(chaves))
    for atr in atributos:
        array[ chaves[atr] ] = 1
    return array

# Converte o RDD para o formato binário, utilizando o dict chaves
binRDD = catPointsRDD.map(lambda rec: (rec[0],CreateNP(rec[1], chaves)))
binRDD.collect()

In [ ]:
# EXERCICIO
# Procure dentre os comandos do PySpark, um que consiga fazer o produto cartesiano da base com ela mesma
cartBinRDD = binRDD.<COMPLETAR>

# Aplique um mapa para transformar nossa RDD em uma RDD de tuplas ((id1,id2), (vetor1,vetor2))
# DICA: primeiro utilize o comando take(1) e imprima o resultado para verificar o formato atual da RDD
cartBinParesRDD = cartBinRDD.<COMPLETAR>


# Aplique um mapa para calcular a Distância de Hamming e Jaccard entre os pares
hamRDD = cartBinParesRDD.<COMPLETAR>
jacRDD = cartBinParesRDD.<COMPLETAR>

# Encontre a distância máxima, mínima e média, aplicando um mapa que transforma (chave,valor) --> valor
# e utilizando os comandos internos do pyspark para o cálculo da min, max, mean
statHRDD = hamRDD.<COMPLETAR>
statJRDD = jacRDD.<COMPLETAR>

Hmin, Hmax, Hmean = statHRDD.<COMPLETAR>, statHRDD.<COMPLETAR>, statHRDD.<COMPLETAR>
Jmin, Jmax, Jmean = statJRDD.<COMPLETAR>, statJRDD.<COMPLETAR>, statJRDD.<COMPLETAR>

print ("\t\tMin\tMax\tMean")
print ("Hamming:\t{:.2f}\t{:.2f}\t{:.2f}".format(Hmin, Hmax, Hmean ))
print ("Jaccard:\t{:.2f}\t{:.2f}\t{:.2f}".format( Jmin, Jmax, Jmean ))

In [ ]:
assert (Hmin.round(2), Hmax.round(2), Hmean.round(2)) == (0.00,5.00,2.40), 'valores incorretos'
print ("OK")
assert (Jmin.round(2), Jmax.round(2), Jmean.round(2)) == (0.60,4.00,1.90), 'valores incorretos'
print ("OK")